In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import json
import re 
import time
from pandas.io.json import json_normalize

# 1. Reading in the DataFrames:
**challengergame** is the players information. 

**ingame** is the overall game informations.

In [2]:
challengergame_df = pd.read_csv('../Data/Match_Data/challenger_match.csv')
challengergame_df2 = pd.read_csv('../Data/Match_Data/challenger_match_V2.csv')

ingame_df = pd.read_pickle('../Data/Match_Data/match_data_version2.pickle')

In [3]:
chall_game = pd.concat([challengergame_df,challengergame_df2],axis = 0)
chall_game_unique = chall_game.drop_duplicates('gameId')
ingame_df['gameId'] = ingame_df['gameId'].astype('int')

# 2. Preparing DataFrames for processing:

First we concatinate the game informations of challenger players. 

Then the Dataframe on the unique challenger games which have been played are merged on the gameId.

In [4]:
chall_match = pd.merge(ingame_df,chall_game_unique[['gameId','accountId']], on = 'gameId', how = 'left')\
                                    .drop(columns=['status.message','status.status_code'])\
                                    .dropna(axis=0)\
                                    .reset_index(drop=True)

In [5]:
match_df = chall_match[chall_match["queueId"]==420].copy().reset_index(drop=True)

In [6]:
champs = pd.read_csv("../Data/Champions_Items_Info/champions.csv")

# 3. Functions to properly process the DataFrames and put it into a ML-able DF:

## 3.1 Function to shape the DF towards ML. Picking features and putting them into a new DF:

In [37]:
def preparing_dataframe(df, lower:int, upper:int):
    df_storage = pd.DataFrame()
    for i in range(lower,upper):
        
        ''' Getting macro information on the game: 
            This is the ID of the game and the duration of the game.
        '''
        
        gameId = df["gameId"].iloc[i]
        gameDuration = df["gameDuration"].iloc[i]
        
        ''' 
        Now looking into the teams statistics, where we want to get information on 
        first blood, first tower, first dragon. If this is True for team "Blue", 
        the value will be set to 1, otherwise it will be set to 0
        '''
        
        blueWins = np.where(df['teams'].iloc[i][0]['win']=="Win", 1, 0)
        firstblood_blue = np.where(df['teams'].iloc[i][0]['firstBlood']==True,1,0)
        firstTower_blue = np.where(df['teams'].iloc[i][0]['firstTower']==True,1,0)
        firstDragon_blue = np.where(df['teams'].iloc[i][0]['firstDragon']==True,1,0)     
        
        '''
        Now taking the number of objectives which have been taken by the team:
        '''
        
        for j in range(len(df['teams'].iloc[i])):
            if df["teams"].iloc[i][j]['teamId']==100:
                dragons_blue = df['teams'].iloc[i][j]['dragonKills']
                barons_blue = df['teams'].iloc[i][j]['baronKills']
                towers_blue = df['teams'].iloc[i][j]['towerKills']
                inhibs_blue = df['teams'].iloc[i][j]['inhibitorKills']
            else:
                dragons_red = df['teams'].iloc[i][j]['dragonKills']
                barons_red = df['teams'].iloc[i][j]['baronKills']
                towers_red = df['teams'].iloc[i][j]['towerKills']
                inhibs_red = df['teams'].iloc[i][j]['inhibitorKills']
            
        
        '''
        Now looking into the different participants
        In there we want to get information on average gold/minute and exp/minute for the early game
        '''
        champion_list_blue = []
        champion_list_red = []
        
        gold_per_minute_blue = 0
        xp_per_minute_blue = 0
        cs_per_minute_blue = 0
        damage_taken_per_minute_blue = 0

        gold_per_minute_red = 0
        xp_per_minute_red = 0
        cs_per_minute_red = 0
        damage_taken_per_minute_red = 0
        
        
        '''blue team : 100'''
        wardplace_blue = 0
        wardkill_blue = 0
        champkill_blue = 0
        champdeath_blue = 0
        champassist_blue = 0
        damage_blue = 0
        gold_blue = 0
        minion_blue = 0
        level_blue = 0
        jungleminion_blue = 0
        killingspree_blue = 0
        heal_blue = 0
        objectdamage_blue = 0

        '''red team : 200'''
        wardplace_red = 0
        wardkill_red = 0
        champkill_red = 0
        champdeath_red = 0
        champassist_red = 0
        damage_red = 0
        gold_red = 0
        minion_red = 0
        level_red = 0
        jungleminion_red = 0
        killingspree_red = 0
        heal_red = 0
        objectdamage_red = 0
        for j in range(len(chall_match['participants'].iloc[i])):

            if chall_match['participants'].iloc[i][j]['teamId'] == 100:
                try:
                    wardplace0 = chall_match['participants'].iloc[i][j]['stats']['wardsPlaced']
                except:
                    wardplace0 = 0
                try:
                    wardkill0 = chall_match['participants'].iloc[i][j]['stats']['wardsKilled']
                except:
                    wardkill0 = 0
                try:
                    champkill0 = chall_match['participants'].iloc[i][j]['stats']['kills']
                except:
                    champkill0 = 0
                try:
                    champdeath0 = chall_match['participants'].iloc[i][j]['stats']['deaths']
                except:
                    champdeath0 = 0
                try:
                    champassist0 = chall_match['participants'].iloc[i][j]['stats']['assists']
                except:
                    champassist0 = 0
                try:
                    damage0 = chall_match['participants'].iloc[i][j]['stats']['totalDamageDealtToChampions']
                except:
                    damage0 = 0
                try:
                    gold0 = chall_match['participants'].iloc[i][j]['stats']['goldEarned']
                except:
                    gold0 = 0
                try:
                    minion0 = chall_match['participants'].iloc[i][j]['stats']['totalMinionsKilled']
                except:
                    minion0 = 0
                try:
                    level0 = chall_match['participants'].iloc[i][j]['stats']['champLevel']
                except:
                    level0 = 0
                try:
                    jungleminion0 = chall_match['participants'].iloc[i][j]['stats']['neutralMinionsKilled']
                except:
                    jungleminion0 = 0
                try:
                    killingspree0 = chall_match['participants'].iloc[i][j]['stats']['killingSprees']
                except:
                    killingspree0 = 0
                try:
                    heal0 = chall_match['participants'].iloc[i][j]['stats']['totalHeal']
                except:
                    heal0 = 0
                try:
                    objectdamage0 = chall_match['participants'].iloc[i][i]['stats']['damageDealtToObjectives']
                except:
                    objectdamage0 = 0

                wardplace_blue += wardplace0
                wardkill_blue += wardkill0
                champkill_blue += champkill0
                champdeath_blue += champdeath0
                champassist_blue += champassist0
                damage_blue += damage0
                gold_blue += gold0
                minion_blue += minion0
                level_blue += level0
                jungleminion_blue += jungleminion0
                killingspree_blue += killingspree0
                heal_blue += heal0
                objectdamage_blue += objectdamage0
            else:

                try:
                    wardplace0 = chall_match['participants'].iloc[i][j]['stats']['wardsPlaced']
                except:
                    wardplace0 = 0

                try:
                    wardkill0 = chall_match['participants'].iloc[i][j]['stats']['wardsKilled']
                except:
                    wardkill0 = 0
                try:
                    champkill0 = chall_match['participants'].iloc[i][j]['stats']['kills']
                except:
                    champkill0 = 0
                try:
                    champdeath0 = chall_match['participants'].iloc[i][j]['stats']['deaths']
                except:
                    champdeath0 = 0
                try:
                    champassist0 = chall_match['participants'].iloc[i][j]['stats']['assists']
                except:
                    champassist0 = 0
                try:
                    damage0 = chall_match['participants'].iloc[i][j]['stats']['totalDamageDealtToChampions']
                except:
                    damage0 = 0
                try:
                    gold0 = chall_match['participants'].iloc[i][j]['stats']['goldEarned']
                except:
                    gold0 = 0
                try:
                    minion0 = chall_match['participants'].iloc[i][j]['stats']['totalMinionsKilled']
                except:
                    minion0 = 0
                try:
                    level0 = chall_match['participants'].iloc[i][j]['stats']['champLevel']
                except:
                    level0 = 0
                try:
                    jungleminion0 = chall_match['participants'].iloc[i][j]['stats']['neutralMinionsKilled']
                except:
                    jungleminion0 = 0
                try:
                    killingspree0 = chall_match['participants'].iloc[i][j]['stats']['killingSprees']
                except:
                    killingspree0 = 0
                try:
                    heal0 = chall_match['participants'].iloc[i][j]['stats']['totalHeal']
                except:
                    heal0 = 0
                try:
                    objectdamage0 = chall_match['participants'].iloc[i][j]['stats']['damageDealtToObjectives']
                except:
                    objectdamage0 = 0


                wardplace_red += wardplace0
                wardkill_red += wardkill0
                champkill_red += champkill0
                champdeath_red += champdeath0
                champassist_red += champassist0
                damage_red += damage0
                gold_red += gold0
                minion_red += minion0
                level_red += level0
                jungleminion_red += jungleminion0
                killingspree_red += killingspree0
                heal_red += heal0
                objectdamage_red += objectdamage0


        
        for j in range(len(df["participants"].iloc[i])):
            if df["participants"].iloc[i][j]["teamId"]==100:
                champion_list_blue.append(df["participants"].iloc[i][j]["championId"])
                try:
                    gpm = df["participants"].iloc[i][j]["timeline"]["goldPerMinDeltas"]["0-10"]
                except:
                    gpm = 0
                try:
                    xpm = df["participants"].iloc[i][j]["timeline"]["xpPerMinDeltas"]["0-10"]
                except:
                    xpm = 0
                try:
                    cspm = df["participants"].iloc[i][j]["timeline"]["creepsPerMinDeltas"]["0-10"]
                except:
                    cspm = 0
                try:
                    dtpm = df["participants"].iloc[i][j]["timeline"]["damageTakenPerMinDeltas"]["0-10"]
                except:
                    dtpm = 0
                    
                gold_per_minute_blue += gpm
                xp_per_minute_blue += xpm
                cs_per_minute_blue += cspm
                damage_taken_per_minute_blue += dtpm 
            else:
                champion_list_red.append(df["participants"].iloc[i][j]["championId"])
                try:
                    gpm = df["participants"].iloc[i][j]["timeline"]["goldPerMinDeltas"]["0-10"]
                except:
                    gpm = 0
                try:
                    xpm = df["participants"].iloc[i][j]["timeline"]["xpPerMinDeltas"]["0-10"]
                except:
                    xpm = 0
                try:
                    cspm = df["participants"].iloc[i][j]["timeline"]["creepsPerMinDeltas"]["0-10"]
                except:
                    cspm = 0
                try:
                    dtpm = df["participants"].iloc[i][j]["timeline"]["damageTakenPerMinDeltas"]["0-10"]
                except:
                    dtpm = 0
                    
                gold_per_minute_red += gpm
                xp_per_minute_red += xpm
                cs_per_minute_red += cspm
                damage_taken_per_minute_red += dtpm 
        
                
        global_result = [int(gameId), int(gameDuration), int(blueWins), int(firstblood_blue),\
                        int(firstTower_blue), int(firstDragon_blue)]
        
        champion_list = [champion_list_blue, champion_list_red]
        
        blue_data = [gold_per_minute_blue, xp_per_minute_blue,\
                     int(cs_per_minute_blue), int(damage_taken_per_minute_blue),\
                     dragons_blue, int(barons_blue), int(towers_blue),\
                     int(inhibs_blue)]
             
        red_data = [int(gold_per_minute_red), int(xp_per_minute_red),\
                    int(cs_per_minute_red), int(damage_taken_per_minute_red),\
                    int(dragons_red), int(barons_red), int(towers_red),\
                    int(inhibs_red)]
        
        num_blue = [wardplace_blue, wardkill_blue, champkill_blue ,champdeath_blue,\
                     champassist_blue ,damage_blue ,gold_blue,minion_blue,level_blue,\
                     jungleminion_blue ,killingspree_blue ,heal_blue ,objectdamage_blue]
        
        num_red = [wardplace_red ,wardkill_red ,champkill_red ,champdeath_red,\
                   champassist_red ,damage_red,gold_red,\
                   minion_red ,level_red ,jungleminion_red,\
                   killingspree_red ,heal_red ,objectdamage_red]
        
        all_data = np.array([global_result+ champion_list+ blue_data+red_data+num_blue+num_red])
        
        columns_list = ["gameId", "gameDuration", "blueWins","First Blood",\
                        "First Tower", "First Dragon","champion_list_blue",\
                        "champion_list_red", "gpm_blue", "xp_pm_blue",\
                        "cs_pm_blue", "dmg_taken_blue","dragons_blue", "barons_blue",\
                        "towers_blue", "inhibs_blue", "gpm_red","xp_pm_red",\
                        "cs_pm_red", "dmg_taken_red", "dragons_red", "barons_red",\
                        "towers_red", "inhibs_red",\
                        "wardplace_blue", "wardkill_blue", "champkill_blue" ,"champdeath_blue",\
                        "champassist_blue" ,"damage_blue" ,"gold_blue","minion_blue","level_blue",\
                        "jungleminion_blue" ,"killingspree_blue" ,"heal_blue" ,"objectdamage_blue",\
                        "wardplace_red" ,"wardkill_red" ,"champkill_red" ,"champdeath_red",\
                        "champassist_red" ,"damage_red","gold_red",\
                        "minion_red" ,"level_red" ,"jungleminion_red",\
                        "killingspree_red" ,"heal_red" ,"objectdamage_red"]
        
        row_df = pd.DataFrame(all_data, columns = columns_list)
        
        df_storage = df_storage.append(row_df)
    return df_storage

## 3.2 Processing the DF:
As this takes a while and if you take too many weight with you, it will kill your computer (if you dont have the proper specs to work with large data).

Therefore, the read in will be put into a function to take the DF and piece by piece put it in the prepared df.

In [38]:
prepared_df = pd.DataFrame()

In [39]:
def processing_df(df, splitter=1):
    '''
    '''
    prepared_df = pd.DataFrame()
    stepsize = len(df)/splitter
    for i in range(splitter):
        lower = int(i*stepsize)
        upper = int((i+1)*stepsize)
        prepared_df = prepared_df.append(preparing_dataframe(df,lower,upper))
        print(i)
    return prepared_df

In [40]:
processed_df = processing_df(match_df,splitter=13)

0
1
2
3
4
5
6
7
8
9
10
11
12


In [98]:
processed_df.head()

,gameId,gameDuration,blueWins,First Blood,First Tower,First Dragon,champion_list_blue,champion_list_red,gpm_blue,xp_pm_blue,...,damage_red,gold_red,minion_red,level_red,jungleminion_red,killingspree_red,heal_red,objectdamage_red,champion_names_blue,champion_names_red
0,4247263043,1323,0,1,0,0,"[7, 350, 266, 517, 110]","[235, 81, 127, 84, 421]",1237,1608.3,...,73016,49832,533,64,138,8,38711,60463,"[LeBlanc, Yuumi, Aatrox, Sylas, Varus]","[Senna, Ezreal, Lissandra, Akali, Rek'Sai]"
0,4247155821,1317,1,0,0,1,"[236, 245, 350, 110, 57]","[235, 126, 517, 58, 15]",1199.7,1674.1,...,44218,37136,571,59,134,3,24829,14105,"[Lucian, Ekko, Yuumi, Varus, Maokai]","[Senna, Jayce, Sylas, Renekton, Sivir]"
0,4243963257,932,0,0,0,0,"[235, 60, 223, 112, 58]","[12, 429, 7, 64, 82]",1311.7,1617.8,...,30797,35364,400,50,89,5,12589,45201,"[Senna, Elise, Tahm Kench, Viktor, Renekton]","[Alistar, Kalista, LeBlanc, Lee Sin, Mordekaiser]"
0,4241678498,2098,0,0,0,1,"[111, 142, 236, 78, 98]","[875, 41, 21, 154, 157]",1318.4,1891.5,...,125866,70528,832,84,205,9,65312,88202,"[Nautilus, Zoe, Lucian, Poppy, Shen]","[Sett, Gangplank, Miss Fortune, Zac, Yasuo]"
0,4241538868,2344,1,1,1,1,"[145, 875, 236, 34, 60]","[62, 2, 517, 142, 350]",1497.2,1863.2,...,131536,74596,794,85,227,7,86431,64141,"[Kai'Sa, Sett, Lucian, Anivia, Elise]","[Wukong, Olaf, Sylas, Zoe, Yuumi]"


In [99]:
processed_df["champion_names_blue"]=0

In [100]:
processed_df["champion_names_red"]=0

In [101]:
for i in range(len(processed_df)):
    processed_df["champion_names_blue"].iloc[i] = [champs[champs["key"]==champion].iloc[0,1]\
                                                  for champion in processed_df["champion_list_blue"].iloc[i]]
    processed_df["champion_names_red"].iloc[i] = [champs[champs["key"]==champion].iloc[0,1]\
                                                  for champion in processed_df["champion_list_red"].iloc[i]]
    if i%2000==0:
        print(i)

/Users/andreasglassl/opt/miniconda3/envs/backpack_banana/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000


In [102]:
processed_df.head()

,gameId,gameDuration,blueWins,First Blood,First Tower,First Dragon,champion_list_blue,champion_list_red,gpm_blue,xp_pm_blue,...,damage_red,gold_red,minion_red,level_red,jungleminion_red,killingspree_red,heal_red,objectdamage_red,champion_names_blue,champion_names_red
0,4247263043,1323,0,1,0,0,"[7, 350, 266, 517, 110]","[235, 81, 127, 84, 421]",1237,1608.3,...,73016,49832,533,64,138,8,38711,60463,"[LeBlanc, Yuumi, Aatrox, Sylas, Varus]","[Senna, Ezreal, Lissandra, Akali, Rek'Sai]"
0,4247155821,1317,1,0,0,1,"[236, 245, 350, 110, 57]","[235, 126, 517, 58, 15]",1199.7,1674.1,...,44218,37136,571,59,134,3,24829,14105,"[Lucian, Ekko, Yuumi, Varus, Maokai]","[Senna, Jayce, Sylas, Renekton, Sivir]"
0,4243963257,932,0,0,0,0,"[235, 60, 223, 112, 58]","[12, 429, 7, 64, 82]",1311.7,1617.8,...,30797,35364,400,50,89,5,12589,45201,"[Senna, Elise, Tahm Kench, Viktor, Renekton]","[Alistar, Kalista, LeBlanc, Lee Sin, Mordekaiser]"
0,4241678498,2098,0,0,0,1,"[111, 142, 236, 78, 98]","[875, 41, 21, 154, 157]",1318.4,1891.5,...,125866,70528,832,84,205,9,65312,88202,"[Nautilus, Zoe, Lucian, Poppy, Shen]","[Sett, Gangplank, Miss Fortune, Zac, Yasuo]"
0,4241538868,2344,1,1,1,1,"[145, 875, 236, 34, 60]","[62, 2, 517, 142, 350]",1497.2,1863.2,...,131536,74596,794,85,227,7,86431,64141,"[Kai'Sa, Sett, Lucian, Anivia, Elise]","[Wukong, Olaf, Sylas, Zoe, Yuumi]"


In [103]:
working_df = processed_df.copy()

In [104]:
working_df["champion_types_blue"]=0
working_df["champion_types_red"]=0

In [105]:
for i in range(len(working_df)):
    working_df["champion_types_blue"].iloc[i] = [eval(champs[champs["key"]==champion].iloc[0,2])\
                                                for champion in working_df["champion_list_blue"].iloc[i]]
    working_df["champion_types_red"].iloc[i] = [eval(champs[champs["key"]==champion].iloc[0,2])\
                                                for champion in working_df["champion_list_red"].iloc[i]]
    if i%2000==0:
        print(i)

/Users/andreasglassl/opt/miniconda3/envs/backpack_banana/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000


In [106]:
working_df.head()

,gameId,gameDuration,blueWins,First Blood,First Tower,First Dragon,champion_list_blue,champion_list_red,gpm_blue,xp_pm_blue,...,minion_red,level_red,jungleminion_red,killingspree_red,heal_red,objectdamage_red,champion_names_blue,champion_names_red,champion_types_blue,champion_types_red
0,4247263043,1323,0,1,0,0,"[7, 350, 266, 517, 110]","[235, 81, 127, 84, 421]",1237,1608.3,...,533,64,138,8,38711,60463,"[LeBlanc, Yuumi, Aatrox, Sylas, Varus]","[Senna, Ezreal, Lissandra, Akali, Rek'Sai]","[[Assassin, Mage], [Support, Mage], [Fighter, ...","[[Marksman, Support], [Marksman, Mage], [Mage]..."
0,4247155821,1317,1,0,0,1,"[236, 245, 350, 110, 57]","[235, 126, 517, 58, 15]",1199.7,1674.1,...,571,59,134,3,24829,14105,"[Lucian, Ekko, Yuumi, Varus, Maokai]","[Senna, Jayce, Sylas, Renekton, Sivir]","[[Marksman], [Assassin, Fighter], [Support, Ma...","[[Marksman, Support], [Fighter, Marksman], [Ma..."
0,4243963257,932,0,0,0,0,"[235, 60, 223, 112, 58]","[12, 429, 7, 64, 82]",1311.7,1617.8,...,400,50,89,5,12589,45201,"[Senna, Elise, Tahm Kench, Viktor, Renekton]","[Alistar, Kalista, LeBlanc, Lee Sin, Mordekaiser]","[[Marksman, Support], [Mage, Fighter], [Suppor...","[[Tank, Support], [Marksman], [Assassin, Mage]..."
0,4241678498,2098,0,0,0,1,"[111, 142, 236, 78, 98]","[875, 41, 21, 154, 157]",1318.4,1891.5,...,832,84,205,9,65312,88202,"[Nautilus, Zoe, Lucian, Poppy, Shen]","[Sett, Gangplank, Miss Fortune, Zac, Yasuo]","[[Tank, Fighter], [Mage, Support], [Marksman],...","[[Fighter, Tank], [Fighter], [Marksman], [Tank..."
0,4241538868,2344,1,1,1,1,"[145, 875, 236, 34, 60]","[62, 2, 517, 142, 350]",1497.2,1863.2,...,794,85,227,7,86431,64141,"[Kai'Sa, Sett, Lucian, Anivia, Elise]","[Wukong, Olaf, Sylas, Zoe, Yuumi]","[[Marksman], [Fighter, Tank], [Marksman], [Mag...","[[Fighter, Tank], [Fighter, Tank], [Mage, Assa..."


In [107]:
working_df.shape

(22711, 54)

In [108]:
champs_list = []
for i in range(len(champs)):
    [champs_list.append(tag) for tag in eval(champs["tags"].loc[i]) if tag not in champs_list]

In [109]:
champs_list

['Fighter', 'Tank', 'Mage', 'Assassin', 'Support', 'Marksman']

# Creating a Column to count each type of champion + the team 

In [110]:
for item in champs_list:
    working_df[item+"_blue"]=0
    working_df[item+"_red"]=0

In [111]:
for i in range(len(working_df)):
    '''
    First process the blue champion types:
    '''
    for j in range(len(working_df["champion_types_blue"].iloc[i])):
        if "Fighter" in working_df["champion_types_blue"].iloc[i][j]:
            working_df["Fighter_blue"].iloc[i]+=1
            
        if "Tank" in working_df["champion_types_blue"].iloc[i][j]:
            working_df["Tank_blue"].iloc[i]+=1
            
        if "Mage" in working_df["champion_types_blue"].iloc[i][j]:
            working_df["Mage_blue"].iloc[i]+=1
            
        if "Assasin" in working_df["champion_types_blue"].iloc[i][j]:
            working_df["Assasin_blue"].iloc[i]+=1
            
        if "Support" in working_df["champion_types_blue"].iloc[i][j]:
            working_df["Support_blue"].iloc[i]+=1
            
        if "Marksman" in working_df["champion_types_blue"].iloc[i][j]:
            working_df["Marksman_blue"].iloc[i]+=1
            
    '''
    Then do the same with the red champions
    '''
    for j in range(len(working_df["champion_types_red"].iloc[i])):
        if "Fighter" in working_df["champion_types_red"].iloc[i][j]:
            working_df["Fighter_red"].iloc[i]+=1
            
        if "Tank" in working_df["champion_types_red"].iloc[i][j]:
            working_df["Tank_red"].iloc[i]+=1
            
        if "Mage" in working_df["champion_types_red"].iloc[i][j]:
            working_df["Mage_red"].iloc[i]+=1
            
        if "Assasin" in working_df["champion_types_red"].iloc[i][j]:
            working_df["Assasin_red"].iloc[i]+=1
            
        if "Support" in working_df["champion_types_red"].iloc[i][j]:
            working_df["Support_red"].iloc[i]+=1
            
        if "Marksman" in working_df["champion_types_red"].iloc[i][j]:
            working_df["Marksman_red"].iloc[i]+=1


/Users/andreasglassl/opt/miniconda3/envs/backpack_banana/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [88]:
#working_df = working_df.drop(columns=["champion_list_blue","champion_list_red"])

In [112]:
working_df=working_df.reset_index(drop=True)

In [113]:
working_df["First Blood"]=working_df["First Blood"].astype(int)

In [114]:
working_df["First Tower"]=working_df["First Tower"].astype(int)

In [115]:
working_df["First Dragon"]=working_df["First Dragon"].astype(int)

In [116]:
working_df["blueWins"]=working_df["blueWins"].astype(int)

In [117]:
working_df.dtypes

gameId           object
gameDuration     object
blueWins          int64
First Blood       int64
First Tower       int64
                  ...  
Assassin_red      int64
Support_blue      int64
Support_red       int64
Marksman_blue     int64
Marksman_red      int64
Length: 66, dtype: object

In [118]:
working_df.to_csv("../Data/Match_Data/prepared_dataset.csv", index=False)

In [96]:
df = pd.read_csv("../Data/Match_Data/prepared_dataset.csv")

In [97]:
df.head()

,gameId,gameDuration,blueWins,First Blood,First Tower,First Dragon,champion_list_blue,champion_list_red,gpm_blue,xp_pm_blue,...,Tank_blue,Tank_red,Mage_blue,Mage_red,Assassin_blue,Assassin_red,Support_blue,Support_red,Marksman_blue,Marksman_red
0,4247263043,1323,0,1,0,0,"[7, 350, 266, 517, 110]","[235, 81, 127, 84, 421]",1237.0,1608.3,...,1,0,4,2,0,0,1,1,1,2
1,4247155821,1317,1,0,0,1,"[236, 245, 350, 110, 57]","[235, 126, 517, 58, 15]",1199.7,1674.1,...,1,1,3,1,0,0,1,1,2,3
2,4243963257,932,0,0,0,0,"[235, 60, 223, 112, 58]","[12, 429, 7, 64, 82]",1311.7,1617.8,...,2,1,2,1,0,0,2,1,1,1
3,4241678498,2098,0,0,0,1,"[111, 142, 236, 78, 98]","[875, 41, 21, 154, 157]",1318.4,1891.5,...,3,2,1,0,0,0,1,0,1,1
4,4241538868,2344,1,1,1,1,"[145, 875, 236, 34, 60]","[62, 2, 517, 142, 350]",1497.2,1863.2,...,1,2,2,3,0,0,1,2,2,0


In [ ]:
num_cols = ["gpm_blue", "xp_pm_blue", "cs_pm_blue", "dmg_taken_blue",\
            "gpm_red", "xp_pm_red", "cs_pm_red", "dmg_taken_red"]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std = StandardScaler()
scaled_fit = std.fit(working_df[num_cols])

In [ ]:
scaled = pd.DataFrame(scaled_fit.transform(working_df[num_cols]), columns=num_cols)

working_df = working_df.drop(columns=num_cols, axis=1)
working_df = working_df.merge(scaled, left_index=True, right_index=True, how="inner")

In [ ]:
train_cols=working_df.columns.to_list()

In [ ]:
train_cols.remove("blueWins")

In [ ]:
train_cols.remove("gameDuration")

In [ ]:
"blueWins" in train_cols

In [ ]:
target_col = "blueWins"

In [ ]:
train_cols.remove("gameId")

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(working_df, test_size=.2, random_state=3)

train_X = train[train_cols].reset_index(drop=True)
train_y = train[target_col].reset_index(drop=True)
test_X = test[train_cols].reset_index(drop=True)
test_y = test[target_col].reset_index(drop=True)

In [ ]:
train_X.dtypes

In [ ]:
def evaluate_model(train_X, train_y, test_X, test_y, model):
    ml_model = model
    ml_model.fit(train_X, train_y)
    
    predictions = ml_model.predict(test_X)
    acc = accuracy_score(test_y, predictions)
    
    print(f"Our model achieved an accuracy of {acc}")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
evaluate_model(train_X, train_y, test_X, test_y, LogisticRegression())

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [ ]:
evaluate_model(train_X, train_y, test_X, test_y, DecisionTreeClassifier())

In [ ]:
evaluate_model(train_X, train_y, test_X, test_y, RandomForestClassifier(n_estimators=100))

In [ ]:
evaluate_model(train_X, train_y, test_X, test_y, SGDClassifier())

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_validate

In [ ]:
models = {"Logistic Regression": LogisticRegression(),
          "Decision Tree": DecisionTreeClassifier(random_state=42),
          "Random Forest": RandomForestClassifier(n_estimators=100, n_jobs=-1)}

In [ ]:
def validate_model(model, X):
    validation_results = cross_validate(model,
                                        X,
                                        train_y.values.ravel(),
                                        cv=5,
                                        scoring="balanced_accuracy")
    acc = validation_results["test_score"].mean()
    print(f"Balanced Mean Accuracy Score: {acc}")

In [ ]:
def evaluate_models(models, X):
    for key, value in models.items():
        print(f"Model: {key}")
        validate_model(value, X)
        print("--------------\n")

In [ ]:
evaluate_models(models, train_X)

In [ ]:
selector_rf = RFECV(models["Random Forest"],
                    cv=5,
                    scoring="balanced_accuracy")

In [ ]:
selector_rf.fit(train_X, train_y)

In [ ]:
selector_rf.get_support()

In [ ]:
train_X.columns

In [ ]:
train_X2 = train_X[train_X.columns[selector_rf.get_support()]]

In [ ]:
evaluate_models(models, train_X2)

In [ ]:
models

In [ ]:
models["Random Forest"] = RandomForestClassifier(n_estimators=500, n_jobs=-1)

In [ ]:
selector_rf2 = RFECV(models["Random Forest"],
                     cv=5,
                     scoring="balanced_accuracy")
selector_rf.fit(train_X, train_y)
train_X3 = train_X[train_X.columns[selector_rf.get_support()]]

In [ ]:
evaluate_models(models, train_X3)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid_rf = {"n_estimators": [500, 1000],
                 "criterion": ["gini", "entropy"],
                 "bootstrap": [True, False],
                 "max_depth": [3, 10, None],
                 "min_samples_leaf": [10, 30, 50, None]}

tune_rf = GridSearchCV(RandomForestClassifier(),
                       param_grid=param_grid_rf,
                       cv=3,
                       scoring="balanced_accuracy",
                       n_jobs=-1)

tune_rf.fit(train_X3, train_y)

In [ ]:
tune_rf.best_params_

In [ ]:
validate_model(RandomForestClassifier(**tune_rf.best_params_), train_X3)

In [ ]:

train_X3 = train_X[train_X.columns[selector_rf.get_support()]]
rf_model = RandomForestClassifier(**tune_rf.best_params_)
rf_model.fit(train_X3, train_y)

test_X3 = test_X[test_X.columns[selector_rf.get_support()]]
rf_pred = rf_model.predict(test_X3)

In [ ]:
evaluate_models(models, train_X3)

In [ ]:
gameId             object
gameDuration       object
blueWins            int64
First Blood         int64
First Tower         int64
First Dragon        int64
Fighter_blue        int64
Fighter_red         int64
Tank_blue           int64
Tank_red            int64
Mage_blue           int64
Mage_red            int64
Assassin_blue       int64
Assassin_red        int64
Support_blue        int64
Support_red         int64
Marksman_blue       int64
Marksman_red        int64
gpm_blue          float64
xp_pm_blue        float64
cs_pm_blue        float64
dmg_taken_blue    float64
gpm_red           float64
xp_pm_red         float64
cs_pm_red         float64
dmg_taken_red     float64
dtype: object

In [ ]:
num_cols = ["Fighter_blue", "Fighter_red", "Tank_blue","Tank_red",\
            "Mage_blue","Mage_red","Assassin_blue","Assassin_red",\
            "Support_blue","Support_red", "Marksman_blue", "Marksman_red",\
            "First Blood", "First Tower"]

In [ ]:
std = StandardScaler()
scaled_fit = std.fit(working_df[num_cols])

In [ ]:
scaled = pd.DataFrame(scaled_fit.transform(working_df[num_cols]), columns=num_cols)

working_df = working_df.drop(columns=num_cols, axis=1)
working_df = working_df.merge(scaled, left_index=True, right_index=True, how="inner")

In [ ]:
train_cols=working_df.columns.to_list()

In [ ]:
working_df.dtypes

In [ ]:
train_cols = num_cols.copy()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(working_df, test_size=.2, random_state=3)

train_X = train[train_cols].reset_index(drop=True)
train_y = train[target_col].reset_index(drop=True)
test_X = test[train_cols].reset_index(drop=True)
test_y = test[target_col].reset_index(drop=True)

In [ ]:
evaluate_model(train_X, train_y, test_X, test_y, LogisticRegression())

In [ ]:
evaluate_models(models, train_X)